<a href="https://colab.research.google.com/github/AlkaidCheng/GSoC2021_QMLHEP/blob/main/TFQ_benchmark_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.4.1
!pip install tensorflow-quantum
!pip install quple

     |████████████████████████████████| 394.3MB 44kB/s 
     |████████████████████████████████| 3.8MB 29.8MB/s 
     |████████████████████████████████| 471kB 56.3MB/s 
     |████████████████████████████████| 2.9MB 38.9MB/s 
  Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0
     |████████████████████████████████| 7.8MB 15.0MB/s 
     |████████████████████████████████| 102kB 13.4MB/s 
     |██████████████████

     |████████████████████████████████| 102kB 11.8MB/s 


In [2]:
import tensorflow as tf
import quple
import cirq
import time
from quple.interface.tfq.layers.pqc import PQC

In [3]:
feature_dimension = n_qubit = 10

In [4]:
from quple.data_encoding import EncodingCircuit
from quple import PauliBlock
data_circuit = EncodingCircuit(feature_dimension=feature_dimension, copies=2, rotation_blocks=['H', 'RZ'], entanglement_blocks=[PauliBlock('ZZ')], entangle_strategy="alternate_linear", 
                               encoding_map="cosine_product",parameter_scale=1)
data_circuit

(0, 0): ───H───Rz(pi*<x_0/pi>)───@───────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────H───Rz(pi*<x_0/pi>)───@─────────────────────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                 │                                   │                                                                                                                                                                                                                                                                                     │                                                 │
(0, 1): ───H───Rz(pi*<x_1/pi>)───X───Rz(pi*<cos(x_0)*cos(x_1)/pi>)───X───────────────────────────────────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────────────────H───Rz(pi*<x_1/pi>)───X─────────────────Rz(pi*<cos(x_0)*cos(x_1)/pi>)───X───────────────────────────────────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                     │                                                                                               │                                                                                                                                                                                                                                       │                                                                                               │
(0, 2): ───H───Rz(pi*<x_2/pi>)───@───────────────────────────────────────────────────────────────────@───────────────────────────────X───────────────────────────────────────────────────────────────Rz(pi*<cos(x_1)*cos(x_2)/pi>)───X───────────────────────────────────────────────────────────────────────────────────────────────H───Rz(pi*<x_2/pi>)───@─────────────────────────────────────────────────────────────────────────────────@───────────────────────────────X───────────────────────────────────────────────────────────────Rz(pi*<cos(x_1)*cos(x_2)/pi>)───X───────────────────────────────────────────────────────────────────────────────────────────────────
                                 │                                                                   │                                                                                                                                                                                                                                                     │                                                                                 │
(0, 3): ───H───Rz(pi*<x_3/pi>)───X───────────────────────────────────Rz(pi*<cos(x_2)*cos(x_3)/pi>)───X───────────────────────────────────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────────────────@───────────────────────────────────────────────────────────────H───Rz(pi*<x_3/pi>)───X─────────────────────────────────────────────────Rz(pi*<cos(x_2)*cos(x_3)/pi>)───X───────────────────────────────────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────────────────@───────────────────────────────────────────────

In [5]:
from quple import ParameterisedCircuit
model_circuit = ParameterisedCircuit(n_qubit=n_qubit, copies=2, rotation_blocks=['RZ', 'RY'], entanglement_blocks=['ZZ','XX'], entangle_strategy="alternate_linear")
model_circuit

(0, 0): ───Rz(θ_0)───Ry(θ_10)───ZZ──────────────────────XX──────────Rz(θ_38)────Ry(θ_48)──────────────ZZ──────────────────────XX──────────────────────
                                │                       │                                             │                       │
(0, 1): ───Rz(θ_1)───Ry(θ_11)───ZZ^(θ_20)───ZZ──────────XX^(θ_29)───XX──────────Rz(θ_39)───Ry(θ_49)───ZZ^(θ_58)───ZZ──────────XX^(θ_67)───XX──────────
                                            │                       │                                             │                       │
(0, 2): ───Rz(θ_2)───Ry(θ_12)───ZZ──────────ZZ^(θ_25)───XX──────────XX^(θ_34)───Rz(θ_40)───Ry(θ_50)───ZZ──────────ZZ^(θ_63)───XX──────────XX^(θ_72)───
                                │                       │                                             │                       │
(0, 3): ───Rz(θ_3)───Ry(θ_13)───ZZ^(θ_21)───ZZ──────────XX^(θ_30)───XX──────────Rz(θ_41)───Ry(θ_51)───ZZ^(θ_59)───ZZ──────────XX^(θ_68)───XX──────────
                                            │                       │                                             │                       │
(0, 4): ───Rz(θ_4)───Ry(θ_14)───ZZ──────────ZZ^(θ_26)───XX──────────XX^(θ_35)───Rz(θ_42)───Ry(θ_52)───ZZ──────────ZZ^(θ_64)───XX──────────XX^(θ_73)───
                                │                       │                                             │                       │
(0, 5): ───Rz(θ_5)───Ry(θ_15)───ZZ^(θ_22)───ZZ──────────XX^(θ_31)───XX──────────Rz(θ_43)───Ry(θ_53)───ZZ^(θ_60)───ZZ──────────XX^(θ_69)───XX──────────
                                            │                       │                                             │                       │
(0, 6): ───Rz(θ_6)───Ry(θ_16)───ZZ──────────ZZ^(θ_27)───XX──────────XX^(θ_36)───Rz(θ_44)───Ry(θ_54)───ZZ──────────ZZ^(θ_65)───XX──────────XX^(θ_74)───
                                │                       │                                             │                       │
(0, 7): ───Rz(θ_7)───Ry(θ_17)───ZZ^(θ_23)───ZZ──────────XX^(θ_32)───XX──────────Rz(θ_45)───Ry(θ_55)───ZZ^(θ_61)───ZZ──────────XX^(θ_70)───XX──────────
                                            │                       │                                             │                       │
(0, 8): ───Rz(θ_8)───Ry(θ_18)───ZZ──────────ZZ^(θ_28)───XX──────────XX^(θ_37)───Rz(θ_46)───Ry(θ_56)───ZZ──────────ZZ^(θ_66)───XX──────────XX^(θ_75)───
                                │                       │                                             │                       │
(0, 9): ───Rz(θ_9)───Ry(θ_19)───ZZ^(θ_24)───────────────XX^(θ_33)───────────────Rz(θ_47)───Ry(θ_57)───ZZ^(θ_62)───────────────XX^(θ_71)───────────────

In [6]:
readout = [cirq.Z(qubit) for qubit in data_circuit.qubits]

In [7]:
pqc_layer = PQC(model_circuit, data_circuit, readout)
batchsize = 10000
x = tf.random.normal((batchsize, feature_dimension))

In [8]:
start = time.time()
y = pqc_layer(x)
end = time.time()
print("Total time taken: {}s".format(end-start))

Total time taken: 33.94687843322754s
